In [158]:
import cassandra
import pandas as pd
import re
import os
import glob
import numpy as np
import json
import csv
from pathlib import Path


pd.set_option("display.max_rows", 999)
pd.options.display.max_rows = 999
pd.options.display.max_columns = 50

In [159]:
pd.get_option("display.max_rows")
pd.get_option("display.max_columns")

50

In [14]:
print(os.getcwd())

#filepath = Path(os.getcwd()) / 'event_data'
filepath = os.getcwd() + '/event_data'
%ls -U {filepath}

/home/sulman/Documents/cassandra
2018-11-09-events.csv  2018-11-30-events.csv  2018-11-11-events.csv
2018-11-28-events.csv  2018-11-20-events.csv  2018-11-17-events.csv
2018-11-26-events.csv  2018-11-04-events.csv  2018-11-03-events.csv
2018-11-21-events.csv  2018-11-22-events.csv  2018-11-06-events.csv
2018-11-08-events.csv  2018-11-23-events.csv  2018-11-05-events.csv
2018-11-14-events.csv  2018-11-16-events.csv  2018-11-24-events.csv
2018-11-12-events.csv  2018-11-25-events.csv  2018-11-27-events.csv
2018-11-02-events.csv  2018-11-13-events.csv  2018-11-07-events.csv
2018-11-29-events.csv  2018-11-19-events.csv  2018-11-15-events.csv
2018-11-18-events.csv  2018-11-01-events.csv  2018-11-10-events.csv


In [21]:
for root, d, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root, '*'))


In [92]:
df_list = []

for f in file_path_list:
    df_tmp = pd.read_csv(f, sep=',')
    df_tmp = df_tmp.dropna(subset=['artist'])
    df_list.append(df_tmp)

df_all_events = pd.concat(df_list)
df_all_events.length = df_all_events.length.apply(lambda v: int(float(v)))
df_all_events.sessionId = df_all_events.sessionId.apply(int)
df_all_events.itemInSession = df_all_events.itemInSession.apply(int)
print(df_all_events.columns)
print(df_all_events.shape)
print(df_all_events.dtypes)

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userId'],
      dtype='object')
(6820, 17)
artist            object
auth              object
firstName         object
gender            object
itemInSession      int64
lastName          object
length             int64
level             object
location          object
method            object
page              object
registration     float64
sessionId          int64
song              object
status             int64
ts               float64
userId           float64
dtype: object


In [93]:
# write csv file to write later to cassandra
df_all_events.iloc[:, [0,2,3,4,5,6,7,8,12,13,16]].to_csv("event_datafile_new.csv", index=False, encoding='utf8')
even_file_path = Path(os.getcwd()) / "event_datafile_new.csv"

In [94]:
even_file_path.exists()
%ls -l .

total 1212
-rw-r--r-- 1 sulman sulman  15860 May 24 11:32 'Cassandra work.ipynb'
drwxr-xr-x 2 sulman sulman   4096 May 22 02:04  event_data/
-rw-rw-r-- 1 sulman sulman 510097 May 22 02:03  event_data-20190522T010324Z-001.zip
-rw-r--r-- 1 sulman sulman 684503 May 24 11:34  event_datafile_new.csv
-rw-rw-r-- 1 sulman sulman  10422 May 24 10:40  lesson-3-demo-1-2-queries-2-tables.ipynb


In [155]:
df_all_events_csv = pd.read_csv("event_datafile_new.csv")
df_all_events_csv.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Muse,Harper,M,1,Barrett,209,paid,"New York-Newark-Jersey City, NY-NJ-PA",275,Supermassive Black Hole (Twilight Soundtrack V...,42.0
1,Beastie Boys,Harper,M,2,Barrett,161,paid,"New York-Newark-Jersey City, NY-NJ-PA",275,Lighten Up,42.0
2,Shakira,Harper,M,3,Barrett,145,paid,"New York-Newark-Jersey City, NY-NJ-PA",275,Pienso En Ti,42.0
3,Selena,Harper,M,4,Barrett,172,paid,"New York-Newark-Jersey City, NY-NJ-PA",275,Amor Prohibido,42.0
4,Kid Cudi Vs Crookers,Harper,M,5,Barrett,162,paid,"New York-Newark-Jersey City, NY-NJ-PA",275,Day 'N' Nite,42.0


In [160]:
df_gp = df_all_events_csv.groupby(['userId', 'song']).size()
#df_gp[df_gp > 1]
#df_all_events.sort_values()#
df_all_events.loc[df_all_events.userId==16][['userId', 'song', 'sessionId', 'itemInSession']].sort_values(by='song')
#df_all_events.loc[df_all_events_csv.groupby(['sessionId', 'itemInSession'])['length'].idxmin()]

,userId,song,sessionId,itemInSession
127,16.0,'Till I Collapse,315,1
184,16.0,(There Was A) Tall Oak Tree,983,39
268,16.0,A Lion's Heart,983,66
307,16.0,A New Arrangement,1076,22
103,16.0,A Song For Sleeping (LP Version),575,13
109,16.0,A Woman's Love (LP Version),983,10
83,16.0,Again,983,3
64,16.0,All Summer Long (Album Version),15,1
333,16.0,All The Right Moves,1076,33
255,16.0,Almaz,983,60


In [45]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

In [46]:
session.hosts

[<Host: 127.0.0.1 datacenter1>]

In [69]:
def try_execute(stmnt, values=None):
    try:
        rows = session.execute(stmnt, values)
        return rows
    except Exception as e:
        print(e)

In [70]:
# create keyspace
try:
    session.execute("""create keyspace if not exists mycassandra
                    with replication = 
                    {'class': 'SimpleStrategy', 'replication_factor':1 }
                    """)
except Exception as e:
    print(e)

In [71]:
# set keyspace
try:
    session.set_keyspace('mycassandra')
except Exception as e:
    print(e)

In [88]:
def write_data(query, values):
# write the data
    assert(isinstance(values, tuple))
    
    event_file  = 'event_datafile_new.csv'

    with open(event_file, 'r', encoding='utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            #query = "insert into music_lib_length (session_id, item_ln_session, artist_name, song_title, song_length)"
            #query += " values(%s, %s, %s, %s, %s)"
            #print(line, '\n')
            rows = try_execute(query, values)
    print(len(rows))
            

In [172]:
# source music data
event_file  = 'event_datafile_new.csv'

## Query 1

In [173]:
# Query 1 - artist, song_title, song_length, that was heard during session '338' and item_session '4'
query_1 = "create table if not exists music_lib_length"
query_1 += "(session_id int, item_in_session int, artist_name text, song_title text, song_length int, PRIMARY KEY(session_id, item_in_session ))"

In [174]:
# create table 1m query 1
try_execute(query_1)

In [175]:
# write the data - query 1

with open(event_file, 'r', encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into music_lib_length (session_id, item_in_session, artist_name, song_title, song_length)"
        query += " values(%s, %s, %s, %s, %s)"
        #print(line, '\n')
        rows = try_execute(query, (int(line[8]), int(line[3]), line[0], line[9], int(float(line[5])), ))
        #print(line[8], line[3], line[0], line[9], int(float(line[5])),)
    print(rows)   

In [177]:
# test query
query = """select artist_name, song_title, song_length from mycassandra.music_lib_length 
        where session_id = 338 and item_in_session = 4;  """

rows = try_execute(query)
for row in rows:
    print(row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495)


## Query 2

In [178]:
# Query 2 - artist, song_title, user_id, first_name, last_name, session_id
query_2 = "create table if not exists music_lib_user"
query_2 += """(user_id int, session_id int, item_in_session int, artist_name text, song_title text, first_name text, last_name text,
               PRIMARY KEY((user_id), session_id, item_in_session ))"""

In [179]:
try_execute(query_2)

In [180]:
# write the data - query 2

with open(event_file, 'r', encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into music_lib_user (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query += " values(%s, %s, %s, %s, %s, %s, %s)"
        #print(line, '\n')
        rows = try_execute(query, (int(float(line[-1])), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        #print(int(line[-1]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4], )
    print(rows)  

In [181]:
# test query
query = """select artist_name, song_title, first_name, last_name from mycassandra.music_lib_user 
        where user_id = 10 and session_id = 182;  """

rows = try_execute(query)
for row in rows:
    print(row)

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


## Query 3

In [182]:
# Query 3 -first_name, last_name, song_title
query_3 = "create table if not exists music_lib_user_song"
query_3 += """(song_title text, user_id int, first_name text, last_name text,
               PRIMARY KEY(song_title, user_id ))"""

In [183]:
try_execute(query_3)

In [184]:
# write the data - query 2

with open(event_file, 'r', encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into music_lib_user_song (song_title, user_id, first_name, last_name)"
        query += " values(%s, %s, %s, %s)"
        #print(line, '\n')
        rows = try_execute(query, (line[9], int(float(line[-1])), line[1], line[4]))
        #print(int(float(line[-1])), line[9], line[1], line[4],)
    print(rows)  

In [186]:
# test query
query = """select first_name, last_name from mycassandra.music_lib_user_song 
where song_title = 'All Hands Against His Own' """

rows = try_execute(query)
for row in rows:
    print(row)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


In [170]:
query_drop_table_1 = "drop table music_lib_length"
query_drop_table_2 = "drop table music_lib_user"
query_drop_table_3 = "drop table music_lib_user_by_song"

In [171]:
try_execute(query_drop_table_1)
try_execute(query_drop_table_2)
try_execute(query_drop_table_3)

Error from server: code=2200 [Invalid query] message="unconfigured table music_lib_user_by_song"


In [ ]:
session.shutdown()
cluster.shutdown()